In [1]:
import utils
import torch
import networkx as nx
import numpy as np
import scipy.sparse as sp
import pandas as pd
from math import ceil
import glob
import unidecode 
from datetime import date, timedelta

In [13]:
os.chdir("../data/Germany")

In [27]:

# TODO i dont think I need these lists (only do it for Germany )
lockdown_indicator = None
lockdown = None

    #------------------------- Germany

print("LOADING GERMANY")
labels = pd.read_csv(f"germany_labels_new_cases_norm.csv")

labels = labels.set_index("name")


sdate = date(2020, 2, 24)
edate = date(2020, 3, 24)
#--- series of graphs and their respective dates
delta = edate - sdate
dates = [sdate + timedelta(days=i) for i in range(delta.days+1)]
dates = [str(date) for date in dates]

labels = labels.loc[:,dates]    #labels.sum(1).values>10

graph_list = utils.generate_graphs_tmp(dates,"GER", "normal")
gs_adj = [nx.adjacency_matrix(kgs).toarray().T for kgs in graph_list]

labels = labels.loc[list(graph_list[0].nodes()),:] # This filters all counties with data that are not in the movement graph

if lockdown:
    lockdown_indicator = pd.read_csv(f"germany_lockdown_{lockdown}.csv")
    lockdown_indicator = lockdown_indicator.set_index("name")
    lockdown_indicator = lockdown_indicator.loc[:,dates]
    lockdown_indicator = lockdown_indicator.loc[list(graph_list[0].nodes()),:]

features = utils.generate_new_features(graph_list ,labels , dates ,8, lockdown_indicator = lockdown_indicator, lockdown = lockdown)

y = list()
for i,G in enumerate(graph_list):
    y.append(list())
    for node in G.nodes():
        y[i].append(labels.loc[node,dates[i]])


LOADING GERMANY


In [24]:
labels

,2020-02-24,2020-02-25,2020-02-26,2020-02-27,2020-02-28,2020-02-29,2020-03-01,2020-03-02,2020-03-03,2020-03-04,...,2020-03-15,2020-03-16,2020-03-17,2020-03-18,2020-03-19,2020-03-20,2020-03-21,2020-03-22,2020-03-23,2020-03-24
name,,,,,,,,,,,,,,,,,,,,,
Landkreis Sonneberg,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.00000,0.000000,0.000000,...,3.506065,7.012131,1.753033,0.000000,1.753033,5.259098,1.753033,0.000000,3.506065,1.753033
Zollernalbkreis,0.0,0.000000,0.526698,0.526698,0.0,0.0,1.580095,3.16019,2.106793,2.633492,...,15.274252,13.694157,20.014537,10.007268,12.114062,11.587363,14.220855,6.320380,13.167458,7.900475
Baden-Baden,0.0,0.000000,0.000000,1.803459,0.0,0.0,0.000000,0.00000,0.000000,3.606918,...,10.820754,9.017295,16.231131,10.820754,3.606918,5.410377,5.410377,5.410377,18.034590,14.427672
Heidelberg,0.0,0.629957,0.000000,0.000000,0.0,0.0,0.000000,0.00000,0.000000,1.259914,...,5.039656,8.189441,12.599139,7.559484,11.339226,3.149785,8.819398,3.779742,8.819398,5.669613
Landkreis Schmalkalden-Meiningen,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.00000,0.000000,0.000000,...,3.219549,2.414662,0.804887,3.219549,1.609775,1.609775,1.609775,1.609775,1.609775,3.219549
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Landkreis Barnim,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.53378,0.000000,0.000000,...,2.135121,3.736462,0.000000,5.871583,1.601341,5.337803,3.202682,1.601341,3.202682,1.067561
Neumünster,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.00000,0.000000,1.251486,...,3.754458,0.000000,2.502972,2.502972,0.000000,3.754458,2.502972,1.251486,1.251486,2.502972
Erfurt,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.00000,0.467963,0.000000,...,0.467963,5.615559,2.339816,4.679632,1.403890,1.871853,1.403890,0.467963,0.935926,0.935926


In [29]:
lockdown_indicator

In [30]:
features[7].shape

(398, 8)

In [69]:
graph_list[1].nodes()

NodeView(('Landkreis Sächsische Schweiz-Osterzgebirge', 'Hamm', 'Salzgitter', 'Landkreis Karlsruhe', 'Landkreis Konstanz', 'Kreis Bergstraße', 'Landkreis Bad Tölz-Wolfratshausen', 'Landkreis Mittelsachsen', 'Landkreis Tirschenreuth', 'Landkreis Tübingen', 'Kempten (Allgäu)', 'Landkreis Potsdam-Mittelmark', 'Landkreis Alzey-Worms', 'Landkreis Regensburg', 'Heilbronn', 'Darmstadt', 'Landkreis Amberg-Sulzbach', 'Landkreis Weißenburg-Gunzenhausen', 'Speyer', 'Landkreis Miesbach', 'Gelsenkirchen', 'Saale-Holzland-Kreis', 'Kreis Olpe', 'Landkreis Cochem-Zell', 'Passau', 'Unstrut-Hainich-Kreis', 'Solingen', 'Landkreis Gießen', 'Freiburg im Breisgau', 'Landkreis Gifhorn', 'Landkreis Schwäbisch Hall', 'Landkreis Mansfeld-Südharz', 'Landkreis Holzminden', 'Kreis Coesfeld', 'Landkreis Osterholz', 'Landkreis Cham', 'Bamberg', 'Ilm-Kreis', 'Landkreis Neu-Ulm', 'Landkreis Rottweil', 'Kreis Herford', 'Landkreis St. Wendel', 'Landkreis Neumarkt in der Oberpfalz', 'Landkreis Oberspreewald-Lausitz', 'La

In [18]:
graph_list

[('Landkreis Sächsische Schweiz-Osterzgebirge',
  'Alb-Donau-Kreis',
  {'weight': 1.71}),
 ('Landkreis Sächsische Schweiz-Osterzgebirge',
  'Altmarkkreis Salzwedel',
  {'weight': 0.43}),
 ('Landkreis Sächsische Schweiz-Osterzgebirge', 'Amberg', {'weight': 0.29}),
 ('Landkreis Sächsische Schweiz-Osterzgebirge', 'Ansbach', {'weight': 3.43}),
 ('Landkreis Sächsische Schweiz-Osterzgebirge',
  'Aschaffenburg',
  {'weight': 0.86}),
 ('Landkreis Sächsische Schweiz-Osterzgebirge', 'Augsburg', {'weight': 1.29}),
 ('Landkreis Sächsische Schweiz-Osterzgebirge',
  'Baden-Baden',
  {'weight': 0.29}),
 ('Landkreis Sächsische Schweiz-Osterzgebirge', 'Bamberg', {'weight': 3.57}),
 ('Landkreis Sächsische Schweiz-Osterzgebirge', 'Bayreuth', {'weight': 8.71}),
 ('Landkreis Sächsische Schweiz-Osterzgebirge', 'Berlin', {'weight': 104.14}),
 ('Landkreis Sächsische Schweiz-Osterzgebirge', 'Bielefeld', {'weight': 1.71}),
 ('Landkreis Sächsische Schweiz-Osterzgebirge', 'Bochum', {'weight': 2.29}),
 ('Landkreis

In [28]:
gs_adj[1].T[0]

array([7.2771e+02, 2.0000e+00, 1.7100e+00, 2.4300e+00, 4.3000e-01,
       1.4300e+00, 1.1400e+00, 1.9986e+02, 4.0000e+00, 1.2900e+00,
       2.9000e-01, 8.8600e+00, 1.1400e+00, 5.5700e+00, 1.0000e+00,
       1.5700e+00, 1.4300e+00, 1.2900e+00, 0.0000e+00, 2.2900e+00,
       1.5700e+00, 2.0140e+01, 1.2900e+00, 5.7000e-01, 7.1000e-01,
       1.5700e+00, 1.1400e+00, 6.2900e+00, 5.7000e-01, 5.1400e+00,
       4.5700e+00, 1.2710e+01, 1.4000e-01, 1.1400e+00, 1.4000e-01,
       2.9000e-01, 3.5700e+00, 1.0140e+01, 2.0000e+00, 7.1000e-01,
       2.8600e+00, 0.0000e+00, 4.4300e+00, 4.1570e+01, 1.4000e-01,
       3.5700e+00, 3.0000e+00, 5.7000e-01, 6.5243e+02, 1.2900e+00,
       2.9000e-01, 2.0000e+00, 4.3000e-01, 1.1000e+01, 7.1000e-01,
       1.1400e+00, 2.1400e+00, 5.4300e+00, 2.2900e+00, 2.9000e-01,
       1.5700e+00, 1.8600e+00, 2.9000e-01, 9.4300e+00, 0.0000e+00,
       8.6000e-01, 1.2900e+00, 1.0000e+00, 1.0000e+00, 1.7100e+00,
       1.0710e+01, 1.0000e+00, 1.1400e+00, 3.2900e+00, 1.1400e

In [46]:
test = sp.block_diag([gs_adj[1].T, gs_adj[1].T])

In [54]:
test2 = utils.sparse_mx_to_torch_sparse_tensor(test)
test2

tensor(indices=tensor([[  0,   1,   2,  ..., 793, 794, 795],
                       [  0,   0,   0,  ..., 795, 795, 795]]),
       values=tensor([7.2771e+02, 1.7100e+00, 1.7100e+00,  ...,
                      1.4000e-01, 1.4000e-01, 1.8614e+02]),
       size=(796, 796), nnz=316808, layout=torch.sparse_coo)

In [61]:
test3 = test2.coalesce().indices()
test3

tensor([[  0,   0,   0,  ..., 795, 795, 795],
        [  0,   1,   2,  ..., 793, 794, 795]])

In [62]:
test2.coalesce().values()

tensor([7.2771e+02, 2.0000e+00, 1.7100e+00,  ..., 8.6000e-01, 1.4000e-01,
        1.8614e+02])

In [58]:
from torch_sparse import SparseTensor
from torch import Tensor
isinstance(test2, Tensor)

True

In [59]:
i = [[0, 1, 1],
         [2, 0, 2]]
v =  [3, 4, 5]
s = torch.sparse_coo_tensor(i, v, (2, 3))
isinstance(test2, SparseTensor)

False

In [29]:
#### Example of Computing Degree Matrix
import numpy as np
matrix = np.matrix('1, 1, 0, 1;'
                   '1, 1, 0, 1;'
                   '0, 0, 1, 1;'
                   '1, 1, 1, 1')

degree = np.zeros(len(matrix)) # initialize list to hold values of degree

# calculate the sums along rows and sum along columns
colsum = matrix.sum(axis=0)
rowsum = matrix.sum(axis=1)

# loop through matrix and add up all degree connections
for j in range(0, len(matrix)):
    degree[j] = colsum[0,j] + rowsum[j,0]

# get the diagonal entries to correct the for loop oversumming
A = matrix.diagonal()
d = A.flat
diagMat = list(d)

# print the degree of connectivity matrix 
D = np.diag(degree - diagMat)
print(D)

[[5. 0. 0. 0.]
 [0. 5. 0. 0.]
 [0. 0. 3. 0.]
 [0. 0. 0. 7.]]


In [34]:
from scipy.linalg import fractional_matrix_power

D_hat = fractional_matrix_power(
    np.matrix('3, 0, 0, 0;'
                   '0, 3, 0, 0;'
                   '0, 0, 2, 0;'
                   '0, 0, 0, 4'), -0.5)

In [35]:
D_hat

array([[0.57735027, 0.        , 0.        , 0.        ],
       [0.        , 0.57735027, 0.        , 0.        ],
       [0.        , 0.        , 0.70710678, 0.        ],
       [0.        , 0.        , 0.        , 0.5       ]])

In [36]:
np.matmul(D_hat, np.matmul(matrix, D_hat))

matrix([[0.33333333, 0.33333333, 0.        , 0.28867513],
        [0.33333333, 0.33333333, 0.        , 0.28867513],
        [0.        , 0.        , 0.5       , 0.35355339],
        [0.28867513, 0.28867513, 0.35355339, 0.25      ]])

In [84]:
normalizeAdjacency(np.array([[1,2, 3, 2],[2, 1, 5, 4],[3, 5, 1, 3],[2, 4, 3, 1]])*2)==normalizeAdjacency(np.array([[1,2, 3, 2],[2, 1, 5, 4],[3, 5, 1, 3],[2, 4, 3, 1]])*1)

array([[False, False,  True,  True],
       [False,  True,  True, False],
       [ True,  True,  True,  True],
       [ True, False,  True, False]])

In [79]:
normalizeAdjacency(np.array([[1,3, 3, 2],[2, 1, 5, 4],[3, 5, 1, 3],[2, 4, 3, 1]])*1)

array([[0.11111111, 0.28867513, 0.28867513, 0.21081851],
       [0.19245009, 0.08333333, 0.41666667, 0.36514837],
       [0.28867513, 0.41666667, 0.08333333, 0.27386128],
       [0.21081851, 0.36514837, 0.27386128, 0.1       ]])

In [80]:
normalizeAdjacency(np.array([[1,3, 3, 2],[2, 1, 5, 4],[3, 5, 1, 3],[2, 4, 3, 1]])*10)

array([[0.11111111, 0.28867513, 0.28867513, 0.21081851],
       [0.19245009, 0.08333333, 0.41666667, 0.36514837],
       [0.28867513, 0.41666667, 0.08333333, 0.27386128],
       [0.21081851, 0.36514837, 0.27386128, 0.1       ]])

In [71]:
0.28867513==0.28867513

True

In [37]:
def normalizeAdjacency(W):
    """
    NormalizeAdjacency: Computes the degree-normalized adjacency matrix

    Input:

        W (np.array): adjacency matrix

    Output:

        A (np.array): degree-normalized adjacency matrix
    """
    # Check that the matrix is square
    assert W.shape[0] == W.shape[1]
    # Compute the degree vector
    d = np.sum(W, axis = 1)
    # Invert the square root of the degree
    d = 1/np.sqrt(d)
    # And build the square root inverse degree matrix
    D = np.diag(d)
    # Return the Normalized Adjacency
    return D @ W @ D 